In [ ]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv(r"C:\NOT_C\Coursera\IBM\capstone\Spacex.csv")
spacex_df.columns = ['Date', 'Time', 'BoosterVersion', 'LaunchSite', 'Payload', 'PayloadMasskg', 'Orbit'\
                    , 'Customer', 'MissionOutcome', 'LandingOutcome']
spacex_df['Class'] = spacex_df.LandingOutcome.apply(lambda x: x[:7] == 'Success')
max_payload = spacex_df['PayloadMasskg'].max()
min_payload = spacex_df['PayloadMasskg'].min()
spacex_df.head()

In [ ]:
# Create a dash application
app = dash.Dash(__name__)

In [ ]:
# Create an app layout
app.layout = html.Div([html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.P('Launch Site:', style={'font-size': 20}),
                                dcc.Dropdown(id='site-dropdown', 
                                    options = [{'label':'All Sites', 'value':'All Sites'},
                                    {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                    {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                    {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                    {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'}],
                                    value='All Sites',
                                    placeholder="Select a Launch Site here",
                                    searchable=True,
                                    style={'width':'40%'},
                                    clearable=False),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart', figure={})),
                                html.Br(),

                                html.P("Payload Range (Kg):", style={'font-size': 20}),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=0, max=10000,
                                               step=1000, value=[min_payload, max_payload],
                                               marks={i:str(i) + ' kg' for i in range(0, 11000, 1000)}),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart', figure={}))
                       
                                ])


In [ ]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df.copy()
    if entered_site == 'All Sites':
        filtered_df.loc[filtered_df['Class'] == 1]
        fig = px.pie(filtered_df, names='LaunchSite', 
                     title='Total Successful launches from all Sites', template='plotly_dark')
    else:        
        def class_name(x):
            if x == False:
                x = 'Failed'
            else:
                x = 'Launched'
            return x
        # choose enteries of only the specific LaunchSite
        filtered_df = filtered_df.loc[filtered_df['LaunchSite'] == entered_site]
        
        # Rename Class entries: False:Failed, True:Launched
        filtered_df['Class'] = filtered_df['Class'].apply(class_name)
        
        fig = px.pie(filtered_df, names='Class',
                     title='Successful launches from: ' + entered_site, template='plotly_dark')
    return fig



@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='payload-slider', component_property='value'),
               Input(component_id='site-dropdown', component_property='value')])
def get_scatter_plot(payload_range, entered_site):
    df = spacex_df.copy()
    if entered_site == 'All Sites':
        low, high = payload_range
        # filter rows for a specific payload_range chosed (using the slider)
        df = df.loc[(df['PayloadMasskg'] > low) & (df['PayloadMasskg'] < high)]
        fig = px.scatter(df,x='PayloadMasskg', y='Class', color='BoosterVersion',  
                         size = 'PayloadMasskg', hover_data=['PayloadMasskg'], template='plotly_dark')
        # change yaxes tick labels
        fig.update_yaxes(tickvals = [False, True], ticktext=['Failed', 'Launched'])
    else:
        low, high = payload_range
        df = df.loc[df['LaunchSite'] == entered_site]
        # filter rows for a specific payload_range chosed (using the slider)
        df = df.loc[(df['PayloadMasskg'] > low) & (df['PayloadMasskg'] < high)]
        fig = px.scatter(df,x='PayloadMasskg', y='Class', color='BoosterVersion', 
                         size = 'PayloadMasskg', hover_data=['PayloadMasskg'], template='plotly_dark')
        # change yaxes tick labels
        fig.update_yaxes(tickvals = [False, True], ticktext=['Failed', 'Launched'])
        
    return fig

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=False)
